In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import fits
import numpy as np

In [2]:
plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.left'] = plt.rcParams['ytick.labelleft'] = False

In [45]:
# Set the path to your images
image_directory = '/Users/davidchemaly/Data/DSS2_Survey/A3D_Catalogue/fits_r_fov10Re/'
output_pdf_directory = './'

stars_in_fov    = np.array(['IC0560', 'IC1447', 'NGC1681', 'NGC3094', 'NGC3230', 'NGC4078', 'NGC4158', 'NGC4266', 'NGC4459', 'NGC4803', 'PGC016060', 'PGC170172', 'UGC10985', 'NGC3193', 'NGC3524'])
artifact_in_fov = np.array(['NGC3023', 'NGC3107', 'NGC3485', 'NGC3507', 'NGC3512', 'NGC3556', 'NGC3631', 'NGC3683', 'NGC3683A', 'NGC3686', 'NGC3780', 'NGC4258', 'NGC5350', 'NGC5353', 'NGC5371', 'NGC5506', 'NGC5602', 'NGC5560', 'NGC5690', 'NGC5792', 'NGC5854', 'NGC5913', 'PGC058114', 'UGC06402'])
np.savetxt('stars_in_fov.txt', stars_in_fov, fmt='%s')
np.savetxt('artifact_in_fov.txt', artifact_in_fov, fmt='%s')   

image_files = [f.split('.')[0] for f in os.listdir(image_directory) if f.endswith('.fits')]

In [46]:
stars_in_fov = np.loadtxt('stars_in_fov.txt', dtype=str)
artifact_in_fov = np.loadtxt('artifact_in_fov.txt', dtype=str)
remove_im = np.concatenate((stars_in_fov, artifact_in_fov))
keep_im = np.setdiff1d(image_files, remove_im)
np.savetxt('keep_im.txt', keep_im, fmt='%s')
np.savetxt('remove_im.txt', remove_im, fmt='%s')


In [48]:
# Ensure the output directory exists
output_pdf_name = 'bad_r_10Re.pdf'
plot_image = remove_im.copy() #sort()
os.makedirs(output_pdf_directory, exist_ok=True)
output_pdf_path = os.path.join(output_pdf_directory, output_pdf_name)

# Get all .png files from the directory
# image_files = [f for f in os.listdir(image_directory) if f.endswith('.png')]
# image_files = [f for f in os.listdir(image_directory) if f.endswith('.fits')]





# Set up PDF writer
with PdfPages(output_pdf_path) as pdf:

    # Iterate over images in chunks of 25 (5x5)
    for i in range(0, len(plot_image), 25):
        fig, axs = plt.subplots(5, 5, figsize=(15, 15))

        # Flatten axs array and iterate over it and the images simultaneously
        for ax, j in zip(axs.flatten(), range(i, min(i+25, len(plot_image)))):

            # # From .png
            # img_path = os.path.join(image_directory, plot_image[j] + '.png')
            # img = plt.imread(img_path)
            # ax.imshow(img)

            # From .fits
            img_path = os.path.join(image_directory, plot_image[j] + '.fits')
            with fits.open(img_path) as hdul:
                data   = hdul[0].data
            arcsinh_im = np.arcsinh(data + data.mean())
            c = 2
            p1 = np.percentile(arcsinh_im, c)
            p2 = np.percentile(arcsinh_im, 100)
            ax.imshow(np.clip(arcsinh_im,p1,p2), cmap='gray_r')

            ax.set_title(plot_image[j].split('.')[0])
            ax.axis('off')

        # Fill remaining subplots (if any) with white color
        for k in range(j+1, i+25):
            axs.flatten()[k-i].axis('off')

        # Save the current plot to the PDF
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF created successfully: {output_pdf_path}")


PDF created successfully: ./bad_r_10Re.pdf
